# Trying Fabrizio's cool random model race 💥

### (1) Load data, trying the function they gave us in the template

In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.insert(0, './scripts')

from proj1_helpers import *

In [4]:
y_train, tx_train, ids_train = load_csv_data('./data/train.csv')

In [57]:
print(tx_train)

[[ 138.47    51.655   97.827 ...    1.24    -2.475  113.497]
 [ 160.937   68.768  103.235 ... -999.    -999.      46.226]
 [-999.     162.172  125.953 ... -999.    -999.      44.251]
 ...
 [ 105.457   60.526   75.839 ... -999.    -999.      41.992]
 [  94.951   19.362   68.812 ... -999.    -999.       0.   ]
 [-999.      72.756   70.831 ... -999.    -999.       0.   ]]


In [77]:
np.shape(tx_train)

(250000, 30)

In [68]:
print(y_train)

[ 1. -1. -1. ...  1. -1. -1.]


In [67]:
y_test, tx_test, ids_test = load_csv_data('./data/test.csv')

### (2) Get rid of features that have too many NAN-values.

Damn, that sucks .... a data set in numpy array form obvsly doesn't contain column-labels!

Get the column indices of the columns we want to get rid of because they contain too many NAs using pandas.

In [81]:
import pandas as pd

data = pd.read_csv('./data/train.csv.zip')

cols_to_remove = []

for i, col in enumerate(data.columns):
    
    if 'leading' in col or 'DER' in col: 
        
        cols_to_remove.append(i-2) # -2 because when using the 'load_csv_data'-function, 'id' and 'prediction'
                                   # are not in tx (the function returns tx, y and ids separately)

In [79]:
print(cols_to_remove)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 23, 24, 25, 26, 27, 28]


In [82]:
tx_train = np.delete(tx_train, cols_to_remove, 1)  

In [85]:
tx_test = np.delete(tx_test, cols_to_remove, 1) 

In [86]:
np.shape(tx_train)

(250000, 11)

In [87]:
np.shape(tx_test)

(568238, 11)

Nice, only the 11 columns without excessive amounts of NAN-values remain.

### (3) TODO: replace NANs with median/mean, same with outliers that are outside 3 times interquartile range

.... coming up ....

### (4) Determine monomials and build a dictionary with instructions for building them.

In [16]:
def number_monomials(n_variables, degree):
    
    '''Computes the number of terms of order 'degree' that can be built from 'n_variables' '''
    
    a = np.math.factorial(n_variables + degree - 1)
    b = np.math.factorial(degree)
    c = np.math.factorial(n_variables - 1)
    
    return a/(b*c)

In [64]:
print(f'There are {number_monomials(11,1)} monomials of order 1.')
print(f'There are {number_monomials(11,2)} monomials of order 2.')
print(f'There are {number_monomials(11,3)} monomials of order 3.')

print(f'Hence, a feature matrix containing all these monomials would have {11+66+286} columns.')

There are 11.0 monomials of order 1.
There are 66.0 monomials of order 2.
There are 286.0 monomials of order 3.
Hence, a feature matrix containing all these monomials would have 363 columns.


At least when using pandas, such a data set doesn't fit in memory. Instead, set up a dictionary that contains instructions for building each of these columns.
Then, when randomly selecting from these 363 columns the respective column can be built on the fly using the instructions.

E.g., if dict[k] = [(2,2),(7,1)], then that means 'column 2 of the original data set raised to the power of 2 times colum 7 of the original data set raised to the power of 1'.

In [6]:
'''
   D-E-G   =   O-N-E   |    D-E-G   =   T-W-O                  | D-E-G  =  T-H-R-E-E
 0  |  1  | ... |  10  |  11   |  12  | ...           |   76   |
--------------------------------------------------------------- ---------------------
x_1 | x_2 | ... | x_11 | x_1^2 | ...  | x_1*x_2 | ... | x_11^2 | .....


- 11 columns of order 1 terms
- 55 + 11 columns of order 2 terms (binomial(11,2) + 11 (for x_1^2, x_2^2 and so on)) = 66
- for order three: 165 (binomial(11,3) + 11 (for x_1^3, x_2^3, ...) + 110 (for (x_1^2)*x_2, (x_2^2)*x_1), etc) = 286

'''

'\n   D-E-G   =   O-N-E   |    D-E-G   =   T-W-O                  | D-E-G  =  T-H-R-E-E\n 0  |  1  | ... |  10  |  11   |  12  | ...           |   76   |\n--------------------------------------------------------------- ---------------------\nx_1 | x_2 | ... | x_11 | x_1^2 | ...  | x_1*x_2 | ... | x_11^2 | .....\n\n\n- 11 columns of order 1 terms\n- 55 + 11 columns of order 2 terms (binomial(11,2) + 11 (for x_1^2, x_2^2 and so on)) = 66\n- for order three: 165 (binomial(11,3) + 11 (for x_1^3, x_2^3, ...) + 110 (for (x_1^2)*x_2, (x_2^2)*x_1), etc) = 286\n\n'

In [66]:
# creating a dictionary for storing the information about columns so they can be generated on the fly
# I KNOW IT'S SUPER MESSY!!!! BUT AS DONALD KNUTH SAID: 'PREMATURE OPTIMIZATION IS THE ROOT OF ALL EVIL', SO LET'S
# CHECK IF THIS APPROACH IS FRUITFUL AT ALL AND THEN IF IT IS OPTIMIZE LATER ;)

cols = {}

# ------------------------------------------------------------------------------------------------------------------
# DEG = 1

# add terms of deg 1

for i in range(11):
    
    cols[i] = [(i,1)]
    
# ------------------------------------------------------------------------------------------------------------------
    
# DEG = 2    
    
# add terms of deg 2 of the form x_1^2, x_2^2 ...
    
for j in range(11, 22):
    
    cols[j] = [(j-11,2)]
    
# now comes the fun part: add terms of deg 2 of the form x_1*x_2, x_1*x_2 ...

k = 0

for j in range (11):

    for l in range (j+1, 11):
    
        cols[22+k] = [(j,1),(l,1)]
        
        k += 1      
        
# ------------------------------------------------------------------------------------------------------------------
        
# DEG = 3
    
# add terms of deg 3 of the form x_1^3, x_2^3 ...
    
for m in range(77, 77+11):
    
    cols[m] = [(m-77,3)]
    
# add terms of deg 3 of the form x_1^2*x_2, x_1^2*x_3 ...    
    
q = 0
    
for n in range(11):
    
    for r in range(11):
        
        if n != r:
            
            cols[88+q] = cols[11+n] + cols[r]
            
            q += 1
            
# finally, add terms of deg 3 of the form x_1*x_2*x_3 ...

s = 198

for n in range(11):
    
    for r in range(n+1, 11):
        
        for u in range (r+1, 11):
            
            cols[s] = [(n,1),(r,1),(u,1)]
            
            s += 1

In [56]:
# Iterate over key/value pairs in dict and print them
for key, value in cols.items():
    print(key, ' : ', value)

0  :  [(0, 1)]
1  :  [(1, 1)]
2  :  [(2, 1)]
3  :  [(3, 1)]
4  :  [(4, 1)]
5  :  [(5, 1)]
6  :  [(6, 1)]
7  :  [(7, 1)]
8  :  [(8, 1)]
9  :  [(9, 1)]
10  :  [(10, 1)]
11  :  [(0, 2)]
12  :  [(1, 2)]
13  :  [(2, 2)]
14  :  [(3, 2)]
15  :  [(4, 2)]
16  :  [(5, 2)]
17  :  [(6, 2)]
18  :  [(7, 2)]
19  :  [(8, 2)]
20  :  [(9, 2)]
21  :  [(10, 2)]
22  :  [(0, 1), (1, 1)]
23  :  [(0, 1), (2, 1)]
24  :  [(0, 1), (3, 1)]
25  :  [(0, 1), (4, 1)]
26  :  [(0, 1), (5, 1)]
27  :  [(0, 1), (6, 1)]
28  :  [(0, 1), (7, 1)]
29  :  [(0, 1), (8, 1)]
30  :  [(0, 1), (9, 1)]
31  :  [(0, 1), (10, 1)]
32  :  [(1, 1), (2, 1)]
33  :  [(1, 1), (3, 1)]
34  :  [(1, 1), (4, 1)]
35  :  [(1, 1), (5, 1)]
36  :  [(1, 1), (6, 1)]
37  :  [(1, 1), (7, 1)]
38  :  [(1, 1), (8, 1)]
39  :  [(1, 1), (9, 1)]
40  :  [(1, 1), (10, 1)]
41  :  [(2, 1), (3, 1)]
42  :  [(2, 1), (4, 1)]
43  :  [(2, 1), (5, 1)]
44  :  [(2, 1), (6, 1)]
45  :  [(2, 1), (7, 1)]
46  :  [(2, 1), (8, 1)]
47  :  [(2, 1), (9, 1)]
48  :  [(2, 1), (10, 1)]
49  :

It seems correct!! 🙏🏻

### (5) Set up a function for randomly generating feature matrices using the dict

In [110]:
def build_feature(ind):
    
    instructions = cols[ind]
    
    res = np.zeros(tx_test.shape[0])
    
    for col, pow_ in instructions:
        
        res += tx_test[:,col] ** pow_
        
    return res

In [111]:
build_feature(177)

array([ 9713.285136, 31111.990001, 12433.365025, ..., 35588.067904,
       45214.742225,  9482.669641])

In [112]:
def build_random_feature_matrix(num_features):
    
    # set seed here if determinism desired
    
    features = np.random.choice(362, num_features) # random permutation of the 363 columns of the feature matrix
                                                   # defined by the cols-dict
        
    res = np.zeros((tx_test.shape[0], num_features))
    
    for i in range(num_features):
        
        col = build_feature(features[i])
        
        res[:,i] = col
        
    return res

In [115]:
random_feature_matrix = build_random_feature_matrix(8)

In [116]:
np.shape(random_feature_matrix)

(568238, 8)